<a href="https://colab.research.google.com/github/wjdbin217/artistic_AI/blob/main/finetuning_prc_medical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. environment setting


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 라이브러리 설치
!pip install transformers datasets
# 앞에 !를 붙이는 이유는 python 코드가 아니라, 터미널 명령어를 직접 실행하겠다는 뜻

!pip install --upgrade transformers


# 2. model training


### 기본 모델 불러오기 & data set 불러와서 확인

In [3]:
# 1. 라이브러리 임포트
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import torch
import sys
import pandas as pd

# 2. 학습에 앞서 GPU를 사용하고 있는지 check
if not torch.cuda.is_available():
    print("현재 GPU가 설정되어 있지 않습니다. Colab 메뉴에서 런타임 유형 변경 > 하드웨어 가속기 : GPU로 바꿔주세요.")
    sys.exit()


# 3. dataset 불러오기 (CSV)
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Finetuning_prc_medical/dialogues_medical(utf-8).csv")  # 드라이브에서 해당 문서 위치 보고 경로 수정

# 4. HuggingFace Dataset으로 변환
dataset = Dataset.from_pandas(df)

print("\nCSV 파일에서 상위 5개 행 출력\n")
print(df.head())  # 기본적으로 상위 5개 행 출력
print("\nHuggingFace Dataset에서 샘플 출력\n")
print(dataset[:5])  # 처음 5개 샘플 출력
print(df.columns) # column명 출력

# 5. train_dataset & test_dataset으로 나누기
from datasets import DatasetDict

split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]


CSV 파일에서 상위 5개 행 출력

                         prompt  \
0  안녕하세요. 감기 증상이 있어서 진료 예약하려고요.   
1              내일 오전 중으로 가능할까요?   
2                9시 45분에 예약할게요.   
3     이수정이고요, 010-5678-1234입니다.   
4             네, 감사합니다. 내일 뵐게요.   

                                          completion  
0      네, 안녕하세요. 내과 진료 예약 도와드릴게요. 원하시는 날짜나 시간 있으실까요?  
1  네, 확인해보겠습니다. 내일 오전에는 9시 45분이나 11시가 비어 있는데, 어떤 ...  
2  네, 내일 오전 9시 45분 내과 진료로 예약 도와드렸습니다. 성함이랑 연락처 여쭤...  
3  확인됐습니다. 처음 예약이시니 접수 시 신분증 지참해주시고, 진료 10분 전까지 도...  
4                            네, 조심히 오세요. 좋은 하루 보내세요.  

HuggingFace Dataset에서 샘플 출력

{'prompt': ['안녕하세요. 감기 증상이 있어서 진료 예약하려고요.', '내일 오전 중으로 가능할까요?', '9시 45분에 예약할게요.', '이수정이고요, 010-5678-1234입니다.', '네, 감사합니다. 내일 뵐게요.'], 'completion': ['네, 안녕하세요. 내과 진료 예약 도와드릴게요. 원하시는 날짜나 시간 있으실까요?', '네, 확인해보겠습니다. 내일 오전에는 9시 45분이나 11시가 비어 있는데, 어떤 시간이 괜찮으세요?', '네, 내일 오전 9시 45분 내과 진료로 예약 도와드렸습니다. 성함이랑 연락처 여쭤봐도 될까요?', '확인됐습니다. 처음 예약이시니 접수 시 신분증 지참해주시고, 진료 10분 전까지 도착 부탁드릴게요.', '네, 조심히 오세요. 좋은 하루 보내세요.']}
Index(

### training 설정

In [4]:
# 5. 모델과 토크나이저 불러오기
model_name = "gogamza/kobart-base-v2"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# 학습 전에 모델을 GPU에 올리기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 6. 전처리 함수 정의
def preprocess_function(example):
    model_inputs = tokenizer(example["prompt"], max_length=128, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["completion"], max_length=128, truncation=True, padding="max_length")

    # padding된 label은 -100으로 마스킹
    labels["input_ids"] = [
        (l if l != tokenizer.pad_token_id else -100) for l in labels["input_ids"]
    ]
    # labels에 pad 토큰 마스킹이 필요한 이유
    # completion은 길이가 다 달라서 padding으로 짧은 문장은 공백으로 채우게 되어 있음. 이게 pad_token
    # 이 pad도 label로 쓰면 모델이 그걸 예측하려고 하는 문제 발생.

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# train, eval 모두 전처리
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_eval = eval_dataset.map(preprocess_function, batched=True)
# KoBART는 BART 기반이라 token_type_ids가 필요 없다 캄. 그래서 token_type_ids도 제외시켜줘야 됨.
tokenized_train = tokenized_train.remove_columns(["prompt", "completion", "token_type_ids"])
tokenized_eval = tokenized_eval.remove_columns(["prompt", "completion", "token_type_ids"])

# 7. 학습 설정
training_args = TrainingArguments(
    output_dir="./kobart-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=8,
    learning_rate=5e-5,
    logging_steps=10,
    remove_unused_columns=False,  # trainer은 기본적으로 모델에 필요한 컬럼 이외의 것을 제거하려고 함.
    # 그래서 이렇게 하면 prompt, completion 컬럼이 남아있어도 무시하지 않고 그냥 넘어감.
)

# 8. Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

# 9. 파인튜닝 시작
trainer.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


Map:   0%|          | 0/215 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/54 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: wjd_bin217 (wjd_bin217-kyung-hee-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,9.397100
20,4.826500
30,3.218500
40,1.582900
50,0.781300
60,0.459700
70,0.314200
80,0.312700
90,0.261600
100,0.377800


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3685: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=864, training_loss=0.338144450896868, metrics={'train_runtime': 145.5908, 'train_samples_per_second': 11.814, 'train_steps_per_second': 5.934, 'total_flos': 131093338521600.0, 'train_loss': 0.338144450896868, 'epoch': 8.0})

# 3. evaluation

In [5]:
eval_results = trainer.evaluate()
print(f"\n평가 결과:\n{eval_results}")


평가 결과:
{'eval_loss': 0.21931932866573334, 'eval_runtime': 0.954, 'eval_samples_per_second': 56.604, 'eval_steps_per_second': 7.337, 'epoch': 8.0}


# 4. inference

In [6]:
# 테스트할 문장 목록
test_sentences = [
    "안녕하세요. 감기 증상이 있어서 진료 예약하려고요.",
    "내일 오전 중으로 가능할까요?",
    "9시 45분에 예약할게요.",
    "이수정이고요, 010-1234-5678 입니다.",
    "네 감사합니다.",
]

# 모델을 평가 모드로 전환
model.eval()

print("\n=== 테스트 문장 응답 ===")
for sentence in test_sentences:
    # 토크나이징
    encoded = tokenizer(sentence, return_tensors="pt", truncation=True, padding="max_length", max_length=64)
    # token_type_ids 제거 (KoBART에서 필요 없음)
    if 'token_type_ids' in encoded:
        del encoded['token_type_ids']

    # GPU 사용 가능하면 모델/입력 데이터를 GPU로 이동
    encoded = {k: v.to(device) for k, v in encoded.items()}

    # 응답 생성
    output_ids = model.generate(
        **encoded,
        max_length=128,
        num_beams=5,
        length_penalty=1.0,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    # 디코딩 후 출력
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(f"사용자: {sentence}")
    print(f"모델 응답: {response}\n")



=== 테스트 문장 응답 ===
사용자: 안녕하세요. 감기 증상이 있어서 진료 예약하려고요.
모델 응답: 네, 안녕하세요. 내과 진료 예약 도와드릴게요. 원하시는 날짜나 시간 있으실까요?하시나요? 안녕하세요?심히 원하시겠어요, 많이 불편하셨겠네요, 안녕하실게요,하려고 하려고 하려고 원하실게요하시네요.드릴하시게 하려고하려고하려고

사용자: 내일 오전 중으로 가능할까요?
모델 응답: 네, 확인해보겠습니다. 내일 오전에는 9시 40분, 10시 50분, 11시 30분 세 타임이 비어 있는데요, 어느 시간대가 괜찮으세요?

사용자: 9시 45분에 예약할게요.
모델 응답: 네, 내일 오전 9시 45분 내과 진료로 예약 도와드렸습니다. 성함이랑 연락처 여쭤봐도 될까요?

사용자: 이수정이고요, 010-1234-5678 입니다.
모델 응답: 네, 예약 완료됐고요. 초진이라 접수 시 신분증 지참하셔야 하고, 10분 전까지 도착 부탁드릴게요. 접수 시간 있으니 10분 일찍 도착해주시면 됩니다. 신분 신분상 진료 시작됐고요, 진료 10분 전에 도착해주실게요? 예약 접수됐고

사용자: 네 감사합니다.
모델 응답: 네, 내일 뵐게요. 쾌유하시길 바랍니다. 쾌유를 빕니다.드릴게요,하실겠습니다.

